In [ ]:
import 

In [ ]:
# Load the data from the CSV file
df = pd.read_csv(r"data\cleansing\filled\data_r_mean_filled.csv")

# Filter the data for dates after 2023
df = df[df["Year"] >2023]

crops = ["Rice", "Wheat", "Atta (Wheat)", "Gram Dal", "Tur/Arhar Dal", "Urad Dal", "Moong Dal", "Masoor Dal", "Sugar", "Milk @", "Groundnut Oil (Packed)", "Mustard Oil (Packed)", "Vanaspati (Packed)", "Soya Oil (Packed)", "Sunflower Oil (Packed)", "Palm Oil (Packed)", "Gur", "Tea Loose", "Salt Pack (Iodised)", "Potato", "Onion", "Tomato"]


In [ ]:
# Create a dictionary to store the last known data for each state
last_known_data = {}
for state in df['States/UTs'].unique():
    last_known_data[state] = df[df['States/UTs'] == state].iloc[-1][crops].values

In [ ]:
# Set look_back period (number of previous days to consider)
look_back = 7

In [ ]:
# Convert date columns to a datetime object
df['Date'] = pd.to_datetime(df[['Day', 'Month', 'Year']])
df.set_index('Date', inplace=True)

In [ ]:
# Load the scaler
scaler = joblib.load('scaler.pkl')

In [ ]:
# Make prediction for 6/9/2024
# First, prepare the input data for 6/9/2024 prediction
last_week_data = df[crops].iloc[-look_back:].values
scaled_last_week = scaler.transform(last_week_data)
reshaped_last_week = np.reshape(scaled_last_week, (1, look_back, len(crops)))

In [ ]:
# Load the model
model = tensorflow.keras.models.load_model('model.keras')

In [ ]:
# Predict
prediction_scaled = model.predict(reshaped_last_week)
prediction = scaler.inverse_transform(prediction_scaled)

In [ ]:
# Create DataFrame for 6/9/2024 predictions
predictions_6_9_2024 = pd.DataFrame(data=prediction, columns=crops, index=['Average Price'])

In [ ]:
# Predict for all states using the last known data
for state, state_data in last_known_data.items():
    scaled_state_data = scaler.transform(state_data.reshape(1,-1))
    reshaped_state_data = np.reshape(scaled_state_data, (1, 1, len(crops)))

    state_prediction_scaled = model.predict(reshaped_state_data)
    state_prediction = scaler.inverse_transform(state_prediction_scaled)

    predictions_6_9_2024.loc[state] = state_prediction[0]

In [ ]:
# Add other metrics (Minimum, Maximum, Modal) based on 6/9/2024 'Average Price' prediction
# Note: Modal price prediction is simplistic and could be improved with a more robust method
predictions_6_9_2024.loc['Maximum Price'] = predictions_6_9_2024.loc['Average Price'] * 1.1
predictions_6_9_2024.loc['Minimum Price'] = predictions_6_9_2024.loc['Average Price'] * 0.9
predictions_6_9_2024.loc['Modal Price']  = predictions_6_9_2024.loc['Average Price'].round()

In [ ]:
# save the predictions to a CSV file
predictions_6_9_2024.to_csv('predictions_6_9_2024.csv')